# Intro

In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import show, PartGroup, Part
from jupyter_cadquery import set_defaults, set_sidecar

set_defaults(axes=True, axes0=True, grid=True)

In [ ]:
import json
import numpy as np
from jupyter_cadquery.cadquery.cad_objects import to_assembly
from jupyter_cadquery.cad_objects import _combined_bb
from jupyter_cadquery import get_default
from jupyter_cadquery.ocp_utils import BoundingBox

preset = lambda key, value: get_default(key) if value is None else value

def convert_assembly(assy, file):
    def _combined_bb(shapes):
        def c_bb(shapes, bb):
            for shape in shapes["parts"]:
                if shape.get("parts") is None:
                    if bb is None:
                        bb = BoundingBox(shape["bb"])
                    else:
                        bb.update(shape["bb"])
                else:
                    return c_bb(shape, bb)
            return bb

        bb = c_bb(shapes, None)
        return bb
    
    def default(obj):
        if type(obj).__module__ == np.__name__:
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            else:
                return obj.item()
        elif isinstance(obj, BoundingBox):
            return obj.to_dict()

        raise TypeError('XXUnknown type:', type(obj))

    part_group = to_assembly(assy)
    mapping = part_group.to_state()
    shapes = part_group.collect_mapped_shapes(
        mapping,
        quality=preset("quality", get_default("quality")),
        deviation=preset("deviation", get_default("deviation")),
        angular_tolerance=preset("angular_tolerance", get_default("angular_tolerance")),
        edge_accuracy=preset("edge_accuracy", get_default("edge_accuracy")),
        render_edges=preset("render_edges", get_default("render_edges")),
        render_normals=preset("render_normals", get_default("render_normals")),
        progress=d.progress,
        timeit=False,
    )
    tree = part_group.to_nav_dict()
    
    data = dict(shapes=shapes, mapping=mapping, tree=tree, bb=_combined_bb(shapes).to_dict())

    with open(f"/tmp/{file}.js", "w") as fd:
        fd.write("const example = ")
        fd.write(json.dumps(data, default=default)) 
        fd.write("\nexport { example }")
        
    return data

In [ ]:
box = cq.Workplane('XY').box(10, 20, 30).edges(">X or <X").fillet(2)
box

# Assemblies

In [ ]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar

set_sidecar("CadQuery", init=True)

In [ ]:
enable_replay(warning=False)  # False disables the repeated warning about replay being enabled!

In [ ]:
box1 = cq.Workplane('XY').box(10, 20, 30).edges(">X or <X").chamfer(2)
box1.name = "box1"

box2 = cq.Workplane('XY').box(8, 18, 28).edges(">X or <X").chamfer(2)
box2.name = "box2"

box3 = cq.Workplane('XY').transformed(offset=(0, 15, 7))\
    .box(30, 20, 6).edges(">Z").fillet(3)
box3.name = "box3"

box4 = box3.mirror("XY").translate((0, -5, 0))
box4.name = "box4"

box1 = box1\
    .cut(box2)\
    .cut(box3)\
    .cut(box4)

In [ ]:
%%time
a1 = PartGroup(
    [
        Part(box1, "red box",   "#d7191c", show_edges=False),
        Part(box3, "green box", "#abdda4", show_edges=False),
        Part(box4, "blue box",  "#2b83ba", show_faces=False),
    ],
    "example 1"
)
d = show(a1, axes=True, grid=True, ortho=False, axes0=True)

# Faces, Edges, Vertices

In [ ]:
box1

In [ ]:
box1.faces("not(|Z or |X or |Y)")

In [ ]:
box1.edges("not(|X or |Y or |Z)")


In [ ]:
box1.vertices()

# Replay

In [ ]:
r = replay(box1)

# Export to HTML

In [ ]:
from ipywidgets.embed import embed_minimal_html, dependency_state
embed_minimal_html(
    'export.html', 
    title='Box', 
    views=[d.cq_view.renderer], 
    state=dependency_state(d.cq_view.renderer))